In [1]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pathlib import Path
from datetime import datetime

#.config("spark.executor.instances", "2")\

spark = SparkSession\
        .builder.master("local[*]")\
        .appName('nyc_taxi')\
        .getOrCreate()

data = [(1,'2019-01-25','20190228',100),
(2,'2018-12-01','20200101',10),
(3,'2019-12-01','20200131',1)]

# Converting string type to date using Python function
datetime.strptime('2019-01-25', '%Y-%m-%d')

schema = StructType([
    StructField('id', StringType()),
    StructField('st_dt', StringType()),
    StructField('end_dt', StringType()),
    StructField('avg_daily_sales', IntegerType())    
])

df = spark.createDataFrame(data, schema)

df.show(truncate=False)
df.createOrReplaceTempView('df')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/29 12:52:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+----------+--------+---------------+
|id |st_dt     |end_dt  |avg_daily_sales|
+---+----------+--------+---------------+
|1  |2019-01-25|20190228|100            |
|2  |2018-12-01|20200101|10             |
|3  |2019-12-01|20200131|1              |
+---+----------+--------+---------------+



In [2]:
# Converting string column in format yyyy-mm-dd to date format from 2 different string type 'yyyy-mm-dd' and 'yyyymmdd'

df2 = df.withColumn("st_dt", to_date(col('st_dt'), 'yyyy-mm-dd'))\
        .withColumn('end_dt', to_date(col('end_dt'), 'yyyymmdd'))
#df2.printSchema()
#df2.show(truncate=False)

Working with date functions in Spark-SQL
    - months_between()
    - year()
    - month()
    - day()
    - date_add()
    - datediff()

In [5]:
query = """
select *,
to_date(st_dt, 'yyyy-mm-dd') as st_dt_date,
months_between(end_dt, st_dt) as months_between_dt,
datediff(end_dt, st_dt) as datediff,
year(st_dt) as year,
month(st_dt) as month,
day(st_dt) as day,
weekofyear(st_dt) as weekofyear,
dayofweek(st_dt) as dayofweek,
dayofmonth(st_dt) as dayofmonth,
next_day(st_dt,"Monday") as next_day,
dayofyear(st_dt) as dayofyear,
dayofmonth(st_dt) as dayofmonth
from df
"""
spark.sql(query).show(truncate=False)

+---+----------+--------+---------------+----------+-----------------+--------+----+-----+---+---------+----------+----------+
|id |st_dt     |end_dt  |avg_daily_sales|st_dt_date|months_between_dt|datediff|year|month|day|dayofweek|weekofyear|dayofmonth|
+---+----------+--------+---------------+----------+-----------------+--------+----+-----+---+---------+----------+----------+
|1  |2019-01-25|20190228|100            |2019-01-25|NULL             |NULL    |2019|1    |25 |6        |4         |25        |
|2  |2018-12-01|20200101|10             |2018-01-01|NULL             |NULL    |2018|12   |1  |7        |48        |1         |
|3  |2019-12-01|20200131|1              |2019-01-01|NULL             |NULL    |2019|12   |1  |1        |48        |1         |
+---+----------+--------+---------------+----------+-----------------+--------+----+-----+---+---------+----------+----------+



Working with date functions in Pyspark dataframe functionality
Pre-requisite : For any of the below mentioned functions to be applied, type should be date
    - months_between()
    - year()
    - month()
    - day()
    - date_add()
    - datediff()
    - dayofweek(), dayofmonth(), dayofyear()

In [6]:
df2.withColumn('months_between', months_between(col('end_dt'), col('st_dt')))\
    .withColumn('datediff', date_diff(col('end_dt'), col('st_dt')))\
    .withColumn('year', year('st_dt'))\
    .withColumn('month', month('st_dt'))\
    .withColumn('day', day('st_dt'))\
    .withColumn('next_day', next_day('st_dt',"Monday"))\
    .withColumn('weekofyear', weekofyear('st_dt'))\
    .withColumn('dayofweek', dayofweek('st_dt'))\
    .withColumn('dayofmonth', dayofmonth('st_dt'))\
    .withColumn('current_timestamp', current_timestamp())\
        .show(truncate=False)

+---+----------+----------+---------------+--------------+--------+----+-----+---+----------+----------+---------+----------+--------------------------+
|id |st_dt     |end_dt    |avg_daily_sales|months_between|datediff|year|month|day|next_day  |weekofyear|dayofweek|dayofmonth|current_timestamp         |
+---+----------+----------+---------------+--------------+--------+----+-----+---+----------+----------+---------+----------+--------------------------+
|1  |2019-01-25|2019-01-28|100            |0.09677419    |3       |2019|1    |25 |2019-01-28|4         |6        |25        |2024-05-29 12:57:16.621077|
|2  |2018-01-01|2020-01-01|10             |24.0          |730     |2018|1    |1  |2018-01-08|1         |2        |1         |2024-05-29 12:57:16.621077|
|3  |2019-01-01|2020-01-31|1              |12.96774194   |395     |2019|1    |1  |2019-01-07|1         |3        |1         |2024-05-29 12:57:16.621077|
+---+----------+----------+---------------+--------------+--------+----+-----+---+

In [ ]:
spark.stop()

In [4]:
query = """
select *
from df
"""
spark.sql(query).show(truncate=False)

+---+----------+--------+---------------+
|id |st_dt     |end_dt  |avg_daily_sales|
+---+----------+--------+---------------+
|1  |2019-01-25|20190228|100            |
|2  |2018-12-01|20200101|10             |
|3  |2019-12-01|20200131|1              |
+---+----------+--------+---------------+

